In [1]:
!pip install -U -q --no-cache-dir ultralytics ipywidgets torchinfo albumentations

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
import shutil

original_folder = '/opt/conda/lib/python3.10/site-packages/ultralytics'
custom_folder = '/kaggle/input/rtdetrv2-ultralytics/ultralytics'

shutil.rmtree(original_folder)

shutil.copytree(custom_folder, original_folder)

'/opt/conda/lib/python3.10/site-packages/ultralytics'

In [3]:
import yaml
import cv2
import numpy as np
import json
import glob
import random
import shutil
import gc
import matplotlib.pyplot as plt
import os
import torch
from tqdm import tqdm
from ultralytics import YOLO, RTDETR

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from torchinfo import summary

In [5]:
torch.cuda.empty_cache()
gc.collect()

0

In [6]:
data_yaml = {
    'train': '/kaggle/input/vehicle-detection-v1/combination/combination/images/train',
    'val': '/kaggle/input/vehicle-detection-v1/combination/combination/images/val',
    'names': ["bus", "car", "motorbike", "truck"],
    'augmentation': {
        'hsv_h': 0.015,  # Hue augmentation (0.0-0.1)
        'hsv_s': 0.7,    # Saturation augmentation (0.0-0.9)
        'hsv_v': 0.4,    # Value augmentation (0.0-0.9)
        'flipud': 0.1,   # Vertical flip probability (0.0-1.0)
        'fliplr': 0.5,   # Horizontal flip probability (0.0-1.0)
        'mosaic': 1.0,   # Enable mosaic augmentation
        'mixup': 0.0,    # Disable mixup
        'scale': 0.5,    # Random scale (0.0-1.0)
        'translate': 0.1, # Random translation (0.0-1.0)
        'perspective': 0.0, # Perspective transformation probability
        'blur': 0.2 # Apply Gaussian blur
    }
}

with open('/kaggle/working/data.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [7]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.3MB/s]


In [8]:
model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [9]:
model.train(data='/kaggle/working/data.yaml', epochs=150, workers=4, batch=128, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1], amp=True, warmup_epochs=20, deterministic=False, lr0=0.01, lrf=0.0001)

New https://pypi.org/project/ultralytics/8.3.96 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=150, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=10, cache=False, device=[0, 1], workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=False, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=

100%|██████████| 755k/755k [00:00<00:00, 16.7MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/vehicle-detection-v1/combination/combination/labels/train... 17064 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17064/17064 [00:49<00:00, 346.31it/s]


train: WARNING ⚠️ Cache directory /kaggle/input/vehicle-detection-v1/combination/combination/labels is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all       4280      38208      0.871      0.851      0.927      0.752

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    150/150      9.19G     0.6782     0.4303     0.9289        157        640: 100%|██████████| 134/134 [02:17<00:00,  1.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:28<00:00,  1.20it/s]


                   all       4280      38208      0.868      0.854      0.927      0.752

150 epochs completed in 7.693 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:34<00:00,  1.02s/it]


                   all       4280      38208      0.873       0.85      0.927      0.752
                   bus       1599       2558      0.877      0.867      0.929       0.75
                   car       3639      20864      0.881      0.876       0.95      0.801
             motorbike       2215      10867       0.88      0.822      0.917      0.709
                 truck       2064       3919      0.852      0.833      0.912      0.749
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


In [10]:
#model.train(data='/kaggle/working/data.yaml', epochs=150, workers=4, batch=128, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1], amp=False, warmup_epochs=20, deterministic=False, lr0=0.01, lrf=0.0001)

In [11]:
# def predictions_from_images_to_labelme(
#         img_path,
#         output_path,
#         label_dict={
#             5: 'bus',
#             2: 'car',
#             3: 'motorbike',
#             7: 'truck'
#         }):
#     """Convert model predictions to LabelMe format

#     Args:
#         model_path (str): path to model
#         video_path (str): path to video
#         output_path (str): path to output folder
#     """
#     os.makedirs(output_path, exist_ok=True)

#     list_imgs = glob.glob(img_path + '/*.jpg')

#     for frame_id, img_path in enumerate(tqdm(list_imgs)):
#         torch.cuda.empty_cache()
#         frame = cv2.imread(img_path)
        
#         results = model.predict(frame, conf=0.5, verbose=False, device='0')[0]
#         class_ids = results.boxes.cls.cpu().numpy()
#         boxes = results.boxes.xyxy.cpu().numpy()

#         annotations = []
#         for class_id, box in zip(class_ids, boxes):
#             if int(class_id) not in label_dict.keys():
#                 continue
            
#             xmin, ymin, xmax, ymax = map(float, box)
#             points = [[xmin, ymin], [xmax, ymax]]
#             annotation = {
#                 "label": label_dict[int(class_id)],
#                 "points": points,
#                 "group_id": None,
#                 "shape_type": "rectangle",
#                 "flags": {}
#             }
#             annotations.append(annotation)

#         labelme_content = {
#             "version": "5.6.0",
#             "flags": {},
#             "shapes": annotations,
#             "imagePath": f"frame_{frame_id}.jpg",
#             "imageData": None,
#             "imageHeight": frame.shape[0],
#             "imageWidth": frame.shape[1]
#         }

#         frame_output_path = os.path.join(output_path, f"frame_{frame_id}.jpg")
#         json_output_path = os.path.join(output_path, f"frame_{frame_id}.json")

#         cv2.imwrite(frame_output_path, frame)
#         with open(json_output_path, 'w') as f:
#             json.dump(labelme_content, f)

In [12]:
# predictions_from_images_to_labelme(
#         img_path="/kaggle/input/vehicle-detection-v1/combination/images/train",
#         output_path="/kaggle/working/combination/train",
#         label_dict={
#             5: 'bus',
#             2: 'car',
#             3: 'motorbike',
#             7: 'truck'
#         })

In [13]:
# predictions_from_images_to_labelme(
#         img_path="/kaggle/input/vehicle-detection-v1/combination/images/val",
#         output_path="/kaggle/working/combination/val",
#         label_dict={
#             5: 'bus',
#             2: 'car',                               
#             3: 'motorbike',
#             7: 'truck'
#         })

In [14]:
# model = YOLO("yolo11n.pt")
# model.train(data='/kaggle/working/data.yaml', epochs=100, workers=16, batch=32, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1])

In [15]:
# model = RTDETR("rtdetr-l.pt")
# model.train(data='/kaggle/working/data.yaml', epochs=100, workers=16, batch=24, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1])

In [16]:
# model = RTDETR("/kaggle/input/rt-detr-continue-to-train/last.pt")
# model.train(data='/kaggle/working/data.yaml', resume=True)                                         